In [1]:
from kafka import KafkaConsumer
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, TimestampType
import json
import pyspark
from pyspark.sql.functions import from_json, col
from datetime import datetime

In [2]:
scala_version = '2.12'
spark_version = '3.2.1'
packages = [
    f'org.apache.spark:spark-sql-kafka-0-10_{scala_version}:{spark_version}',
    'org.apache.kafka:kafka-clients:3.2.1'
]
spark = SparkSession.builder\
   .master("local")\
   .appName("kafka-example")\
   .config("spark.jars.packages", ",".join(packages))\
   .getOrCreate()
#consumer = KafkaConsumer('t1')
#for msg in consumer:
#    print(json.loads(msg.value))

22/12/08 17:01:20 WARN Utils: Your hostname, MSI resolves to a loopback address: 127.0.1.1; using 192.168.124.1 instead (on interface eth1)
22/12/08 17:01:20 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/drahcear/hadoop/spark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/drahcear/.ivy2/cache
The jars for the packages stored in: /home/drahcear/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.kafka#kafka-clients added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-f896b3e0-a9d7-4b82-8bd0-2131ff0ea507;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.2.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.2.1 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.1 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.1 in central
	found org.apache.htrace#htrace-core4;4.1.0-incubating in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
	found org.apache.kafka#kafka-clients

In [3]:
schema = StructType([
  StructField('date', StringType(), True),
  StructField('ID', StringType(), True),
  StructField('country', StringType(), True),
    StructField('name', StringType(), True),
    StructField('number', StringType(), True),
    StructField('CVC', StringType(), True),
    StructField('expire', StringType(), True),
    StructField('IsLegit', StringType(), True),
    StructField('reason', StringType(), True)
  ])

df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "localhost:9092") \
  .option("subscribe", "t4") \
  .option("startingOffsets", "latest") \
  .option("failOnDataLoss", "false") \
  .load() \
  .select(from_json(col("value").cast("string"), schema).alias("json"))\
  .select("json.*")        

In [8]:
df.writeStream \
.trigger(processingTime='5 seconds')\
.option("checkpointLocation", "checkpoint/") \
      .option("path", datetime.now().strftime('%Y-%m-%d'))\
      .option("header","true") \
      .outputMode("append")\
      .format("csv")\
      .start().awaitTermination()

"""
df.writeStream \
.trigger(processingTime='5 seconds')\
      .outputMode("append")\
      .format("console")\
      .start().awaitTermination()
"""

22/12/08 17:07:03 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/home/drahcear/pytorch/lib/python3.8/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/drahcear/pytorch/lib/python3.8/site-packages/py4j/clientserver.py", line 475, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.8/socket.py", line 669, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 